In [0]:
%pip install langchain
dbutils.library.restartPython()

In [0]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read the text file
df = spark.read.text("dbfs:/Volumes/rea-onyx-uc-catalog/rea_onyx_uc_schema/rea_test_volume/physicalasset.csv")

# Collect all the text into a single string
text_column = " ".join([row.value for row in df.collect()])

length_function = len

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=length_function,
)
chunks = splitter.split_text(text_column)

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import ArrayType, StringType
import pandas as pd

@pandas_udf("array<string>")
def get_chunks(dummy):
    return pd.Series([chunks])

# Register the UDF
spark.udf.register("get_chunks_udf", get_chunks)

In [0]:
%sql
insert into `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_text (text)
select explode(get_chunks_udf('dummy')) as text;

In [0]:
# Create a temporary table from the DataFrame
df.createOrReplaceTempView("temp_table")

# Ensure the catalog, schema, and table names are correct
# Adjust the query to use the correct names if they were misspelled
spark.sql("""
    INSERT INTO `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track
    SELECT temp_table.*
    FROM temp_table
    LEFT ANTI JOIN `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track
    ON temp_table.value = `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track.file_name
""")